In [3]:
import requests
from bs4 import BeautifulSoup
import re
from IPython.display import Image
import time

In [4]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [5]:
# Utilizamos Selenium para extraer datos

from selenium.webdriver import Firefox
from selenium.webdriver.firefox.options import Options
opts = Options()
opts.set_headless()
assert opts.headless  # Operating in headless mode
#browser = Firefox(options=opts)

<ipython-input-5-b56b380520ce>:6: DeprecationWarning: use setter for headless property instead of set_headless
  opts.set_headless()


In [114]:
def extract_rarity_value(text):
    text = text.replace(',','')
    data = int(re.findall('(\d+)',text)[0])
    return data
    

In [115]:
types=[]
skin=[]
total_traits =[]
traits =[]
punk_id=[]
rarity = []

total_punks = 10000

for i in range(total_punks):
    try:
        punk_id.append(i)
        browser = Firefox(executable_path = '/home/vant/Descargas/geckodriver-v0.29.1-linux64/geckodriver')
        url = 'https://defypunk.com/punks/'+str(i)
        browser.get(url)
        browser.implicitly_wait(20)

        rars = browser.find_element_by_xpath('/html/body/div/div/main/div/div/div/div[1]/div[2]')

        rarity.append(extract_rarity_value(rars.text))

        items = browser.find_element_by_xpath('/html/body/div/div/main/div/div/div/div[1]')
        # these are the items marked with tag a
        values = items.find_elements_by_tag_name('a')

        types.append(values[0].text)
        skin_property =(values[1].text).split('-')
        skin.append(skin_property[0])
        total_traits.append(values[2].text)
        traits.append(','.join([x.text for x in values[3:]]))
        browser.close()
    except Exception as err:
        print(f'exepcion {err}')
        if len(types) == len(skin) == len(total_traits) == len(traits) == len(punk_id) == len(rarity):
            df = pd.DataFrame({'id':punk_id,'types':types,'rarity':rarity,'skin':skin,'total_traits':total_traits,'traits':traits})
        else:
            print('not all have same length: investigate possible discrepancies')

In [119]:
df = pd.DataFrame({'id':punk_id,'types':types,'rarity':rarity,'skin':skin,'total_traits':total_traits,'traits':traits})
#df.set_index('id',inplace=True)

In [120]:
df.tail()

,id,types,rarity,skin,total_traits,traits
0,0,Female,2023,Mid,3,"Blonde Bob,Earring,Green Eye Shadow,Hidden Ear..."
1,1,Male,4352,Dark,2,"Mohawk,Smile"
2,2,Female,8090,Light,1,Wild Hair
3,3,Male,7896,Dark,3,"Nerd Glasses,Pipe,Wild Hair"
4,4,Male,7332,Mid,4,"Big Shades,Earring,Goat,Wild Hair"


df.to_csv('punks_atributes.csv')

In [1]:
# srapping historic Sales

In [60]:
punk_id =[]
amount_eth = []
amount_dollars = []
transaction_date = []


for j in range(3250,3500):
    try:        
        browser = Firefox(executable_path = '/home/vant/Descargas/geckodriver-v0.29.1-linux64/geckodriver')
        url = 'https://www.larvalabs.com/cryptopunks/details/'+str(j)
        browser.get(url)
        browser.implicitly_wait(10)

        table = browser.find_element_by_tag_name('table')
        #rows = table.find_elements_by_tag_name('td')
        
        rows = table.find_elements_by_class_name('punk-history-row-sold')
        for row in rows:
            punk_id.append(j)
            items = row.text.split(' ')             
            amount_eth.append(items[3])
            amount_dollars.append(items[4])
            transaction_date.append(' '.join(items[5:]))
            
     
        browser.close()
        

        
    except Exception as err:
        print(f'exepcion {err}')
        
df = pd.DataFrame({'id':punk_id,'amount_eth':amount_eth,'amount_dollars':amount_dollars,'transaction':transaction_date})
        

exepcion Message: TypeError: browsingContext.currentWindowGlobal is null

exepcion Message: Unable to locate element: table

exepcion Message: TypeError: browsingContext.currentWindowGlobal is null

exepcion Message: Unable to locate element: table

exepcion Message: Unable to locate element: table



In [61]:
len(punk_id),len(amount_eth),len(amount_dollars),len(transaction_date)

(459, 459, 459, 459)

In [62]:
df = pd.DataFrame({'id':punk_id,'amount_eth':amount_eth,'amount_dollars':amount_dollars,'transaction':transaction_date})

In [63]:
df.head(),df.tail()

(     id amount_eth amount_dollars   transaction
 0  3250        15Ξ      ($25,985)  Feb 15, 2021
 1  3250      5.85Ξ       ($2,053)  Oct 08, 2020
 2  3253      0.40Ξ          ($44)  Nov 27, 2018
 3  3253      0.30Ξ          ($81)  Jul 04, 2017
 4  3254      4.99Ξ       ($1,756)  Sep 27, 2020,
        id amount_eth amount_dollars   transaction
 454  3498        13Ξ      ($25,021)  Feb 19, 2021
 455  3498         9Ξ      ($16,367)  Feb 14, 2021
 456  3498      0.45Ξ          ($78)  May 06, 2019
 457  3499      2.50Ξ         ($821)  Sep 24, 2020
 458  3499      1.99Ξ         ($724)  Aug 02, 2020)

In [64]:
df.to_csv('df3250_3500.csv')